In [1]:
import csv
import numpy as np
X = []
with open("./data/train/crawler/data/tweets.txt.text", newline='', encoding='utf8') as file_data:
    i = 0 
    for row in file_data:
        if i < 500:
            i = i +1
            X.append(row)
            
y = []
with open("./data/train/crawler/data/tweets.txt.labels", newline='', encoding='utf8') as file_data:
    j = 0
    for row in file_data:
        if j < 500:
            j = j +1
            y.append(row.replace("\n",""))


In [2]:
X = np.array(X)
y = np.array(y)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X_dtm = vect.fit_transform(X)

In [4]:
test=[]
with open("./data/test/us_test.text", newline='', encoding='utf8') as test_data:
    file = test_data.readlines()
    for row in file:
        test.append(row.replace("\n",""))
        
test = np.asarray(test)
test_dtm = vect.transform(test)

In [5]:
test_label = []
with open("./data/test/us_test.labels", newline='', encoding='utf8') as test_data_label:
    file = test_data_label.readlines()
    for row in file:
        test_label.append(row.replace("\n",""))
test_label = np.asarray(test_label)
test_label = test_label.reshape(-1,1)

(50000, 1)
(50000,)


In [6]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, f1_score, jaccard_score, classification_report

models = [
    #LogisticRegression(random_state=0, solver='lbfgs',
     #                     multi_class='multinomial',
     #                   n_jobs=-1, max_iter= 1000),
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    RandomForestClassifier(n_estimators=200, random_state=0, criterion='entropy'),
    RandomForestClassifier(n_estimators=200)
    #LinearSVC(),
    #MultinomialNB(),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    model.fit(X_dtm, y)
    predict_label = model.predict(test_dtm)
    acc = accuracy_score(predict_label,test_label)
    f1 = f1_score(predict_label, test_label, average = 'weighted')
    cm = multilabel_confusion_matrix(test_label,predict_label)
    jaccard = jaccard_score(test_label, predict_label, average='micro')
    print(classification_report(test_label, predict_label))
    print ("la matrice de confusion : ")
    print(cm)
    entries.append((model_name, acc, f1, jaccard))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'accuracy', 'f1', 'jaccard'])

/home/julien/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/julien/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.22      1.00      0.36     10798
           1       0.00      0.00      0.00      4830
          10       0.00      0.00      0.00      1432
          11       0.00      0.00      0.00      1949
          12       0.00      0.00      0.00      1265
          13       0.00      0.00      0.00      1114
          14       0.00      0.00      0.00      1306
          15       0.00      0.00      0.00      1244
          16       0.00      0.00      0.00      1153
          17       0.00      0.00      0.00      1545
          18       0.00      0.00      0.00      2417
          19       0.00      0.00      0.00      1010
           2       0.00      0.00      0.00      4534
           3       0.00      0.00      0.00      2605
           4       0.00      0.00      0.00      3716
           5       0.00      0.00      0.00      1613
           6       0.00      0.00      0.00      1996
           7       0.00    

In [7]:
print (cv_df)

               model_name  accuracy        f1   jaccard
0  RandomForestClassifier   0.21596  0.355209  0.121051
1  RandomForestClassifier   0.21646  0.303113  0.121365
2  RandomForestClassifier   0.23320  0.324431  0.131990


In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier

# Adam works better ; Logistic works better
#parameters = {'solver':('lbfgs', 'adam'), 'activation':('relu', 'logistic'), 'alpha':[0.0001, 0.00001, 0.001], 
#              'learning_rate':('adaptive'), }

# Gini works better than entropy and more estimators, better the score is
# parameters = {'n_estimators':[200, 300], 'criterion':('entropy', 'gini')}

# Best with 'alpha': 0.001, 'loss': 'log', 'penalty': 'l2'
parameters = {'loss':('hinge', 'log', 'modified_huber'), 'penalty':('l2', 'elasticnet'), 'alpha':[0.0001, 0.001], }
sgd = SGDClassifier(l1_ratio=0.15, fit_intercept=True, max_iter=1000, 
                    tol=0.001, epsilon=0.1, n_jobs=-1, validation_fraction=0.1, 
                    n_iter_no_change=5)

rdf = RandomForestClassifier()
mlp = MLPClassifier()

clf = GridSearchCV(sgd, parameters, cv=3, n_jobs=-1)

clf.fit(X_dtm, y)
                            
print(clf.cv_results_)